In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt 
import numpy as np
from pylab import cm
import pandas as pd

import matplotlib.font_manager as fm
font_names = [f.name for f in fm.fontManager.ttflist]

mpl.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 22
plt.rcParams['axes.linewidth'] = 2

colors = cm.get_cmap('tab10', 3)
print(colors(1))

from numpy import log2, zeros, mean, var, sum, loadtxt, arange, array, cumsum, dot, transpose, diagonal, sqrt
from numpy.linalg import inv

def block(x): 
    # preliminaries
    n = len(x)
    d = int(log2(n))
    s, gamma = zeros(d), zeros(d)
    mu = mean(x)

    # estimate the auto-covariance and variances 
    # for each blocking transformation
    for i in arange(0,d):
        n = len(x)
        # estimate autocovariance of x
        gamma[i] = (n)**(-1)*sum( (x[0:(n-1)]-mu)*(x[1:n]-mu) )
        # estimate variance of x
        s[i] = var(x)
        # perform blocking transformation
        x = 0.5*(x[0:-1:2] + x[1::2])
   
    # generate the test observator M_k from the theorem
    M = (cumsum( ((gamma/s)**2*2**arange(1,d+1)[::-1])[::-1] )  )[::-1]

    # we need a list of magic numbers
    q =array([6.634897,9.210340, 11.344867, 13.276704, 15.086272, 16.811894, 18.475307, 20.090235, 21.665994, 23.209251, 24.724970, 26.216967, 27.688250, 29.141238, 30.577914, 31.999927, 33.408664, 34.805306, 36.190869, 37.566235, 38.932173, 40.289360, 41.638398, 42.979820, 44.314105, 45.641683, 46.962942, 48.278236, 49.587884, 50.892181])

    # use magic to determine when we should have stopped blocking
    for k in arange(0,d):
        if(M[k] < q[k]):
            break
    if (k >= d-1):
        print("Warning: Use more data")
    return mu, s[k]/2**(d-k)


def acceptance_ratio(e):
    length = len(e)
    accepted =  0
    for i in range(1,length): 
        if e[i] != e[i-1]:
            accepted += 1
    print(accepted/length)
    return accepted/length

(0.5490196078431373, 0.33725490196078434, 0.29411764705882354, 1.0)


In [4]:
is22_1 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_1.0_numMCSteps_262144_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_1 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.1_numMCSteps_262144_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_01 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.01_numMCSteps_262144_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_001 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.001_numMCSteps_262144_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")

is22 = [is22_1, is22_0_1, is22_0_01, is22_0_001]

In [6]:
energies = [block(sample)[0] for sample in is22]
variances = [block(sample)[1] for sample in is22]
acceptances = [acceptance_ratio(sample) for sample in is22]
mc_step_lengths = [1.0, 0.1, 0.01, 0.001]

df = pd.DataFrame({"MC step-length": mc_step_lengths,"Energy" : energies, "Var" : variances, "Acceptance ratio" : acceptances })
print(df.to_latex(index=False))

0.9999961853027344
0.9999961853027344
0.9999961853027344
0.9999961853027344
\begin{tabular}{rrrr}
\toprule
 MC step-length &    Energy &       Var &  Acceptance ratio \\
\midrule
          1.000 &  2.948432 &  0.000008 &          0.999996 \\
          0.100 &  2.996853 &  0.000009 &          0.999996 \\
          0.010 &  3.022168 &  0.000095 &          0.999996 \\
          0.001 &  3.254571 &  0.003392 &          0.999996 \\
\bottomrule
\end{tabular}



In [7]:
is22_1 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_1.0_numMCSteps_1048576_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_1 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.1_numMCSteps_1048576_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_01 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.01_numMCSteps_1048576_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")
is22_0_001 = np.loadtxt("../Data/quantumDot/MC/Interacting/wf_slater_gaussian_nn_sysInfo_is_stepLength_0.001_numMCSteps_1048576_numDims_2_numParticles_2_elementinfo_slater_none_gaussian_none_nn_numhidden1_12_numhidden2_12_activationFunction_tanH_.txt")

is22 = [is22_1, is22_0_1, is22_0_01, is22_0_001]

In [8]:
energies = [block(sample)[0] for sample in is22]
variances = [block(sample)[1] for sample in is22]
acceptances = [acceptance_ratio(sample) for sample in is22]
mc_step_lengths = [1.0, 0.1, 0.01, 0.001]

df = pd.DataFrame({"MC step-length": mc_step_lengths,"Energy" : energies, "Var" : variances, "Acceptance ratio" : acceptances })
print(df.to_latex(index=False))

0.9999990463256836
0.9999990463256836
0.9999990463256836
0.9999990463256836
\begin{tabular}{rrrr}
\toprule
 MC step-length &    Energy &       Var &  Acceptance ratio \\
\midrule
          1.000 &  2.936667 &  0.000002 &          0.999999 \\
          0.100 &  2.990406 &  0.000002 &          0.999999 \\
          0.010 &  3.009645 &  0.000012 &          0.999999 \\
          0.001 &  3.187466 &  0.000660 &          0.999999 \\
\bottomrule
\end{tabular}

